In [1]:
import pandas as pd
import re

In [19]:
import pandas as pd

# Read the CSV file
csv_file_path = r"/Users/dhruvarora/Documents/Smollan : Google/Version_1.csv"
df = pd.read_csv(csv_file_path)

# Column to count lines
column_name = "Detected Text"

# Filter out rows that contain "< Previous" or "Next >"
exclude_patterns = ["< Previous", "Next >"]
filtered_df = df.copy()  # Make a copy to avoid modifying the original DataFrame
for pattern in exclude_patterns:
    filtered_df = filtered_df[~filtered_df[column_name].str.contains(pattern, na=False)]

# Filter out rows where "Deal of the Day" or "Limited time Deal" appears more than 2 times
deal_pattern = "Deal of the Day"
filtered_df = filtered_df[filtered_df[column_name].str.count(deal_pattern) <= 2]

limited_time_deal_pattern = "Limited time Deal"
filtered_df = filtered_df[filtered_df[column_name].str.count(limited_time_deal_pattern) <= 2]

# Remove rows with empty or whitespace-only text
filtered_df = filtered_df[~filtered_df[column_name].str.strip().eq("")]

# Filter out rows where total character count is less than 100
filtered_df = filtered_df[filtered_df[column_name].apply(len) >= 100]

# Reset the index after filtering
filtered_df.reset_index(drop=True, inplace=True)

In [20]:
grouped = filtered_df.groupby("Image Name")

# Function to reset "Order" within each group
def reset_order(group):
    group = group.reset_index(drop=True)  # Reset the index for each group
    group["Order"] = group.index + 1  # Update the "Order" to be sequential
    return group

# Apply the function to each group
df_corrected = grouped.apply(reset_order)

In [21]:
df_corrected.reset_index(drop=True, inplace=True)

In [22]:
df_corrected['Order'] = df_corrected.groupby('Image Name')['Order'].transform(lambda x: len(x) - x + 1)


In [23]:
df_corrected.to_csv(r'/Users/dhruvarora/Desktop/corrected_output.csv')

In [24]:
df = df_corrected

In [25]:
import re
import pandas as pd

# Define regular expression patterns
product_pattern = re.compile(
    r"(.*?)(?=(Limited time deal|Deal of the Day|bought in past month|\.\.\.|(\d{1,3}(,\d{3})*)\s*M\.R\.P|(\d{1,3}(,\d{3})*)\s*✔prime))",
    re.DOTALL
)

a_price_pattern = re.compile(r"\*?(\d{1,3}(,\d{3})*)\*?\s*(?=(M\.R\.P|\.\.\.|\.\.\.))")
b_price_pattern = r"\*?(\d{1,3}(,\d{3})*)\*? M\.R\.P"
discounted_price_pattern = re.compile(f"{a_price_pattern}|{b_price_pattern}")
mrp_pattern = re.compile(r"M\.R\.P:\s*([^ ]+)")
discount_pattern = re.compile(r"\(\d+%\s*off\)")
delivery_by_pattern = re.compile(r"Get it by\s*[^\n]+")

# Extract information from detected text
def extract_info(text):
    product_match = re.search(product_pattern, text)
    product_details = product_match.group().strip() if product_match else None
    
    discounted_price_match = re.search(discounted_price_pattern, text)
    discounted_price = discounted_price_match.group().strip() if discounted_price_match else None

    original_price_match = re.search(mrp_pattern, text)
    original_price = original_price_match.group(1).strip() if original_price_match else None

    discount_match = re.search(discount_pattern, text)
    discount = discount_match.group().strip() if discount_match else None

    delivery_by_match = re.search(delivery_by_pattern, text)
    delivery_by = delivery_by_match.group().strip() if delivery_by_match else None
    
    # Assign "Other Details" only if text is not empty or whitespace
    other_details = text if text.strip() else None

    return {
        "Product": product_details,
        "Discounted Price": discounted_price,
        "Original Price": original_price,
        "Discount": discount,
        "Delivery By": delivery_by,
        "Other Details": other_details,  # Assign only if text is not empty
    }

# Apply the function to create new columns from "Detected Text"
df_extracted = df['Detected Text'].apply(extract_info).apply(pd.Series)

# Assign to original DataFrame with specified columns
df[['Product', 'DiscountedPrice', 'OriginalPrice', 'Discount', 'DeliveryBy', 'OtherDetails']] = df_extracted


In [26]:
df['InterOrder'] = df['Image Name'].str.rsplit('_', n=1).str[-1]

In [27]:
df['InterOrder'] = df['InterOrder'].str.split('.', n=1).str[0]

In [28]:
df.rename(columns = {'Order':'IntraOrder', 'Detected Text': 'RawText', 'Bucket Link': 'ImageLink', 'Product':'ProductName', 'Image Link':'ImageLink', 'Image Name': 'ImageName'}, inplace=True)

In [29]:
df['Listing_Id'] = df['ImageName']+'_'+df['ProductName']+'_'+df['InterOrder']

In [30]:
# Function to extract the 'region' and 'producttype'
def extract_info(value):
    # Extract the region between the first and second underscores
    region_pattern = r"_(.*?)_"  # Look for text between the first and second underscores
    region_match = re.search(region_pattern, value)
    region = region_match.group(1) if region_match else None
    
    # Extract the producttype between a number with an underscore and before '_listing'
    producttype_pattern = r"_\d+_(.*)_listing"  # Match text between a number with underscore and '_listing'
    producttype_match = re.search(producttype_pattern, value)
    producttype = producttype_match.group(1) if producttype_match else None
    
    return pd.Series({
        'Region': region,
        'ProductType': producttype,
    })

# Apply the function to the DataFrame
df[['Region', 'ProductType']] = df['ImageName'].apply(extract_info)

In [31]:
from datetime import datetime
def extract_date_from_url(url):
    # Regular expression to find the date pattern
    date_pattern = re.compile(r'\d{2}-\d{2}-\d{2}')
    
    # Extract the date from the URL
    match = date_pattern.search(url)
    if match:
        # Extracted date string
        date_str = match.group(0)
        
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(date_str, '%d-%m-%y')
        
        # Format the date in SQL date-time format
        sql_date = date_obj.strftime('%Y-%m-%d')
        
        return sql_date
    else:
        return None # Return None if no date is found

# Apply the function to the 'Image Link' column and create a new column 'Date'
df['Date'] = df['ImageLink'].apply(extract_date_from_url)


In [32]:
new_order = ['ImageName', 'IntraOrder', 'RawText', 'ImageLink', 'ProductName', 'DiscountedPrice', 'OriginalPrice', 'Discount', 'DeliveryBy', 'OtherDetails', 'InterOrder', 'Listing_Id', 'Region', 'ProductType', 'Date']
df = df[new_order]

In [33]:
df.to_csv(r'/Users/dhruvarora/Desktop/corrected_output.csv', index=False)

In [34]:
# import pandas as pd
# from sqlalchemy import create_engine
# from urllib.parse import quote_plus

# csv_file = r"/Users/dhruvarora/Desktop/corrected_output.csv"
# user = quote_plus("u781023547_shashwat")
# password = quote_plus("consti@123C")
# host = "srv1326.hstgr.io"
# port = "3306"
# database = "u781023547_harmony_paas"

# connection_string = (
#     f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
# )
# engine = create_engine(connection_string)


# df = pd.read_csv(csv_file)

# df.to_sql('my_table', engine, if_exists='replace', index=False)


858

In [35]:
# # SQL query to be executed
# query = "SELECT * FROM my_table"

# # Read the result of the SQL query into a DataFrame
# newdf = pd.read_sql_query(query, engine)  